In [ ]:
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.synthesis.stabilizer import synth_circuit_from_stabilizers
from qiskit_aer import AerSimulator
import math
from matplotlib.patches import Polygon
import copy
import networkx as nx
from collections import deque
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error
import random
from collections import Counter
import numpy as np

In [ ]:
def inject_pauli_error(circuit: QuantumCircuit, logical_registers: dict, error_probability: float = 0.1) -> QuantumCircuit:
    pauli_errors = ['x','y','z']
    data_qubits = []
    for reg in logical_registers.values():
        data_qubits.extend(reg)
    
    for qubit in circuit.qubits:
        if qubit in data_qubits:
            if random.random() < error_probability:
                error = random.choice(pauli_errors)
                if error == 'x':
                    circuit.x(qubit)
                elif error == 'y':
                    circuit.y(qubit)
                elif error == 'z':
                    circuit.z(qubit)
    return circuit

In [ ]:
def stab_to_binary(stab_str: str) -> str:
    return ''.join('1' if ch == 'X' else '0' for ch in stab_str)

def xor_binary_strings(a: str, b: str) -> str:
    return ''.join('1' if (x != y) else '0' for x, y in zip(a, b))

def flip_first_n_bits(s: str, n: int) -> str:
    flipped = ''.join('1' if c == '0' else '0' for c in s[:n])
    return flipped + s[n:]

def generate_logical0_set_from_xstabs(x_stabs: list) -> set:
    if not x_stabs:
        return set()
    
    bin_generators = [stab_to_binary(s) for s in x_stabs]
    L = len(bin_generators[0])
    
    logical0_set = { "0" * L }
    for vec in bin_generators:
        new_set = set()
        for existing in logical0_set:
            xor_vec = xor_binary_strings(existing, vec)
            new_set.add(xor_vec)
        logical0_set = logical0_set.union(new_set)
    return logical0_set

def generate_logical_basis_sets(x_stabs: list, n: int) -> (set, set):
    logical0_set = generate_logical0_set_from_xstabs(x_stabs)
    logical1_set = { flip_first_n_bits(vec, n) for vec in logical0_set }
    return logical0_set, logical1_set


In [ ]:
def generate_single_grid(n, grid_index, offset=(0,0)):
    ox, oy = offset
    d_points = []
    for y in range(1, 2*n, 2):
        for x in range(1, 2*n, 2):
            d_points.append((x, y))
    d_points_sorted = sorted(d_points, key=lambda pt: (pt[1], pt[0]))
    d_numbering = {pt: i+1 for i, pt in enumerate(d_points_sorted)}
    di_coords = {f"D{grid_index}_{num}": (pt[0] + ox, pt[1] + oy)
                 for pt, num in d_numbering.items()}
    ax_points = []
    for x in range(0, 2*n+1, 2):
        for y in range(2, 2*n, 2):
            if (x + y) % 4 == 0:
                ax_points.append((x, y))
    ax_points_sorted = sorted(ax_points, key=lambda pt: (pt[1], pt[0]))
    ax_numbering = {pt: i+1 for i, pt in enumerate(ax_points_sorted)}
    axi_coords = {f"Ax{grid_index}_{num}": (pt[0] + ox, pt[1] + oy)
                  for pt, num in ax_numbering.items()}
    az_points = []
    for x in range(2, 2*n, 2):
        for y in range(0, 2*n+1, 2):
            if (x + y) % 4 == 2:
                az_points.append((x, y))
    az_points_sorted = sorted(az_points, key=lambda pt: (pt[1], pt[0]))
    az_numbering = {pt: i+1 for i, pt in enumerate(az_points_sorted)}
    azi_coords = {f"Az{grid_index}_{num}": (pt[0] + ox, pt[1] + oy)
                  for pt, num in az_numbering.items()}
    adj_axi = {}
    for pt_ax, ax_num in ax_numbering.items():
        label_ax = f"Ax{grid_index}_{ax_num}"
        adjacent = []
        for pt_d, d_num in d_numbering.items():
            if abs(pt_d[0] - pt_ax[0]) == 1 and abs(pt_d[1] - pt_ax[1]) == 1:
                adjacent.append(f"D{grid_index}_{d_num}")
        adj_axi[label_ax] = adjacent
    adj_azi = {}
    for pt_az, az_num in az_numbering.items():
        label_az = f"Az{grid_index}_{az_num}"
        adjacent = []
        for pt_d, d_num in d_numbering.items():
            if abs(pt_d[0] - pt_az[0]) == 1 and abs(pt_d[1] - pt_az[1]) == 1:
                adjacent.append(f"D{grid_index}_{d_num}")
        adj_azi[label_az] = adjacent
    return di_coords, axi_coords, azi_coords, adj_axi, adj_azi

def plot_multi_grids(n, l, m, d):
    def sort_points(points):
        cx = sum(p[0] for p in points) / len(points)
        cy = sum(p[1] for p in points) / len(points)
        return sorted(points, key=lambda p: math.atan2(p[1]-cy, p[0]-cx))
    
    fig, ax = plt.subplots(figsize=(15,15))
    grid_width = 2 * n
    grid_height = 2 * n
    grids = {}
    
    for row in range(l):
        for col in range(m):
            grid_index = row * m + col + 1
            label_L = f"L_{grid_index}"
            ox = col * (grid_width + d)
            oy = row * (grid_height + d)
            di_coords, axi_coords, azi_coords, adj_axi, adj_azi = generate_single_grid(n, grid_index, offset=(ox, oy))
            grids[label_L] = {"di_coords": di_coords,
                              "axi_coords": axi_coords,
                              "azi_coords": azi_coords,
                              "adj_axi": adj_axi,
                              "adj_azi": adj_azi}
            for key, (x, y) in di_coords.items():
                ax.scatter(x, y, facecolors='none', edgecolors='black', zorder=3, s=100)
                ax.text(x, y, key, color='black', ha='center', va='center', fontsize=10, zorder=4)
            for key, (x, y) in axi_coords.items():
                ax.scatter(x, y, color='yellow', s=80, zorder=2)
                ax.text(x, y, key, color='black', ha='center', va='center', fontsize=8, zorder=4)
            for key, (x, y) in azi_coords.items():
                ax.scatter(x, y, color='green', s=80, zorder=2)
                ax.text(x, y, key, color='black', ha='center', va='center', fontsize=8, zorder=4)
            for ax_label, (ax_x, ax_y) in axi_coords.items():
                adjacent = adj_axi.get(ax_label, [])
                if len(adjacent) == 4:
                    pts = [di_coords[dq_label] for dq_label in adjacent]
                    pts = sort_points(pts)
                    xs, ys = zip(*pts)
                    ax.fill(xs, ys, color='lightgoldenrodyellow', alpha=0.5, zorder=0)
                elif len(adjacent) == 2:
                    pts = [(ax_x, ax_y)] + [di_coords[dq_label] for dq_label in adjacent]
                    pts = sort_points(pts)
                    xs, ys = zip(*pts)
                    ax.fill(xs, ys, color='lightgoldenrodyellow', alpha=0.5, zorder=0)
            for az_label, (az_x, az_y) in azi_coords.items():
                adjacent = adj_azi.get(az_label, [])
                if len(adjacent) == 4:
                    pts = [di_coords[dq_label] for dq_label in adjacent]
                    pts = sort_points(pts)
                    xs, ys = zip(*pts)
                    ax.fill(xs, ys, color='palegreen', alpha=0.5, zorder=0)
                elif len(adjacent) == 2:
                    pts = [(az_x, az_y)] + [di_coords[dq_label] for dq_label in adjacent]
                    pts = sort_points(pts)
                    xs, ys = zip(*pts)
                    ax.fill(xs, ys, color='palegreen', alpha=0.5, zorder=0)
                        
    total_width = m * (grid_width+d) - d
    total_height = l * (grid_height+d) - d
    ax.set_xlim(-1, total_width+1)
    ax.set_ylim(-1, total_height+1)
    ax.set_aspect('equal')
    ax.set_xticks(range(0, total_width+1))
    ax.set_yticks(range(0, total_height+1))
    ax.grid(True, which='both', linestyle='--', color='lightgray')
    plt.title(f"{l} x {m} lattice (n={n}, d={d})")
    plt.show()
    return grids

def generate_stabilizers(grids_info):
    stabilizers = {}
    for grid_label, grid_data in grids_info.items():
        data_qubits = sorted(grid_data["di_coords"].keys(), key=lambda k: int(k.split('_')[1]))
        z_stabs = {}
        for az_label, adj_list in grid_data["adj_azi"].items():
            stab = "".join(["Z" if dq in adj_list else "I" for dq in data_qubits])
            z_stabs[az_label] = stab
        x_stabs = {}
        for ax_label, adj_list in grid_data["adj_axi"].items():
            stab = "".join(["X" if dq in adj_list else "I" for dq in data_qubits])
            x_stabs[ax_label] = stab
        stabilizers[grid_label] = {"z_stabilizers": z_stabs, "x_stabilizers": x_stabs}
    return stabilizers


In [ ]:
# ================= Syndrome Extraction with Deep Copy and Simulation =================

def extract_syndrome(qc: QuantumCircuit,
                     grids_info: dict,
                     logical_registers: dict,
                     x_ancilla_registers: dict,
                     z_ancilla_registers: dict) -> (QuantumCircuit, dict):
    
    qc_sim = copy.deepcopy(qc)
    
    for L_label in sorted(grids_info.keys(), key=lambda k: int(k.split('_')[1])):
        grid_data = grids_info[L_label]
        data_labels = sorted(grid_data["di_coords"].keys(), key=lambda k: int(k.split('_')[1]))
        data_reg = logical_registers[L_label]
        # Z syndrome extraction
        azi_labels = sorted(grid_data["azi_coords"].keys(), key=lambda k: int(k.split('_')[1]))
        z_reg = z_ancilla_registers[L_label]
        for i, az_label in enumerate(azi_labels):
            adj_list = grid_data["adj_azi"][az_label]
            for dq_label in adj_list:
                j = data_labels.index(dq_label)
                qc_sim.cx(data_reg[j], z_reg[i])
        # X syndrome extraction
        axi_labels = sorted(grid_data["axi_coords"].keys(), key=lambda k: int(k.split('_')[1]))
        x_reg = x_ancilla_registers[L_label]
        for i, _ in enumerate(axi_labels):
            qc_sim.h(x_reg[i])
        for i, ax_label in enumerate(axi_labels):
            adj_list = grid_data["adj_axi"][ax_label]
            for dq_label in adj_list:
                j = data_labels.index(dq_label)
                qc_sim.cx(x_reg[i], data_reg[j])
        for i, _ in enumerate(axi_labels):
            qc_sim.h(x_reg[i])
        # Ancilla 
        num_z = len(z_reg)
        creg_z = ClassicalRegister(num_z, name=f"cz_{L_label}")
        qc_sim.add_register(creg_z)
        qc_sim.measure(z_reg, creg_z)
        num_x = len(x_reg)
        creg_x = ClassicalRegister(num_x, name=f"cx_{L_label}")
        qc_sim.add_register(creg_x)
        qc_sim.measure(x_reg, creg_x)
    
    sim = AerSimulator()
    result = sim.run(qc_sim, shots=1).result()
    syndrome = result.get_counts(qc_sim)
    print("Syndrome extraction simulation syndrome:")
    print(syndrome)
    return qc, syndrome


def error_correction(qc, syndrome, grids_info, logical_registers, n, m, verbose=False):
    import networkx as nx
    from collections import deque

    def diagonal_path_between(start, target):
        moves = [(2,2), (2,-2), (-2,2), (-2,-2)]
        queue = deque()
        queue.append((start, [start]))
        visited = {start}
        while queue:
            current, path = queue.popleft()
            if current == target:
                return path, len(path)-1  
            for dx, dy in moves:
                next_node = (current[0] + dx, current[1] + dy)
                if next_node not in visited:
                    visited.add(next_node)
                    queue.append((next_node, path + [next_node]))
        return None, float('inf')  

    corrected_qubits = {}      
    correction_paths = {}       
    ancilla_defects_info = {}  

    syndrome_key = max(syndrome, key=syndrome.get)
    syndrome_clean = syndrome_key.replace(" ", "")[::-1]
    
    total_lattices = len(grids_info)
    total_slices = 2 * total_lattices  
    chunk_length = len(syndrome_clean) // total_slices
    slices = [syndrome_clean[i*chunk_length:(i+1)*chunk_length] for i in range(total_slices)]
    
    if verbose:
        print("각 격자에 할당된 syndrome 슬라이스:")
    lattice_labels = sorted(grids_info.keys(), key=lambda L: int(L.split('_')[1]))
    for idx, L in enumerate(lattice_labels):
        x_slice = slices[2*idx]
        z_slice = slices[2*idx + 1]
        if verbose:
            print(f"{L}: x_slice = {x_slice}, z_slice = {z_slice}")
    
    for idx, L in enumerate(lattice_labels):
        corrected_qubits[L] = {"X_correction": [], "Z_correction": []}
        correction_paths[L] = {"X": [], "Z": [], "unmatched_X": [], "unmatched_Z": []}
        ancilla_defects_info[L] = {"Az_defects": [], "Ax_defects": []}
        
       
        x_slice = slices[2*idx] 
        z_slice = slices[2*idx + 1]  
        
        
        azi_keys = sorted(grids_info[L]["azi_coords"].keys(), key=lambda s: int(s.split('_')[1]))
        x_defects = []
        for i, bit in enumerate(x_slice):
            if bit == '1':
                if i < len(azi_keys):
                    ancilla_label = azi_keys[i]
                    defect_coord = grids_info[L]["azi_coords"][ancilla_label]
                    x_defects.append((ancilla_label, defect_coord))
        ancilla_defects_info[L]["Az_defects"] = x_defects
        
        if len(x_defects) >= 2:
            G = nx.Graph()
            
            for idx_def, (_, coord) in enumerate(x_defects):
                G.add_node(idx_def, coord=coord)
            num_defects = len(x_defects)
           
            for i in range(num_defects):
                for j in range(i+1, num_defects):
                    _, coord_i = x_defects[i]
                    _, coord_j = x_defects[j]
                    path, cost = diagonal_path_between(coord_i, coord_j)
                    G.add_edge(i, j, weight=cost)
            
            matching = nx.algorithms.matching.min_weight_matching(G, weight='weight')
            if verbose:
                print(f"Lattice {L}: X defect 매칭 결과 (MWPM): {matching}")
            
        
            matched_nodes = set()
            for a, b in matching:
                matched_nodes.add(a)
                matched_nodes.add(b)
            all_nodes = set(range(num_defects))
            unmatched_nodes = list(all_nodes - matched_nodes)
            if unmatched_nodes:
                correction_paths[L]["unmatched_X"].extend([x_defects[i][0] for i in unmatched_nodes])
            
            for a, b in matching:
                _, coord_a = x_defects[a]
                _, coord_b = x_defects[b]
                best_path, best_cost = diagonal_path_between(coord_a, coord_b)
                correction_paths[L]["X"].append(best_path)
                
                for i in range(len(best_path) - 1):
                    midpoint = ((best_path[i][0] + best_path[i+1][0]) // 2,
                                (best_path[i][1] + best_path[i+1][1]) // 2)
                    for d_label, d_coord in grids_info[L]["di_coords"].items():
                        if d_coord == midpoint:
                            sorted_data_keys = sorted(grids_info[L]["di_coords"].keys(), key=lambda s: int(s.split('_')[1]))
                            index = sorted_data_keys.index(d_label)
                            qc.x(logical_registers[L][index])
                            corrected_qubits[L]["X_correction"].append(d_label)
                            if verbose:
                                print(f"Lattice {L}: X correction applied on {d_label} at {d_coord} (midpoint of {best_path[i]} and {best_path[i+1]})")
        else:
            if verbose:
                print(f"Lattice {L}: X 오류 보정을 위한 Az defect가 충분하지 않습니다. 감지된 Az defects: {x_defects}")
            correction_paths[L]["unmatched_X"].extend([anc_label for anc_label, _ in x_defects])
        
       
        axi_keys = sorted(grids_info[L]["axi_coords"].keys(), key=lambda s: int(s.split('_')[1]))
        z_defects = []
        for i, bit in enumerate(z_slice):
            if bit == '1':
                if i < len(axi_keys):
                    ancilla_label = axi_keys[i]
                    defect_coord = grids_info[L]["axi_coords"][ancilla_label]
                    z_defects.append((ancilla_label, defect_coord))
        ancilla_defects_info[L]["Ax_defects"] = z_defects
        
        if len(z_defects) >= 2:
            G = nx.Graph()
            for idx_def, (_, coord) in enumerate(z_defects):
                G.add_node(idx_def, coord=coord)
            num_defects = len(z_defects)
            for i in range(num_defects):
                for j in range(i+1, num_defects):
                    _, coord_i = z_defects[i]
                    _, coord_j = z_defects[j]
                    path, cost = diagonal_path_between(coord_i, coord_j)
                    G.add_edge(i, j, weight=cost)
            matching = nx.algorithms.matching.min_weight_matching(G, weight='weight')
            if verbose:
                print(f"Lattice {L}: Z defect 매칭 결과 (MWPM): {matching}")
            
            matched_nodes = set()
            for a, b in matching:
                matched_nodes.add(a)
                matched_nodes.add(b)
            all_nodes = set(range(num_defects))
            unmatched_nodes = list(all_nodes - matched_nodes)
            if unmatched_nodes:
                correction_paths[L]["unmatched_Z"].extend([z_defects[i][0] for i in unmatched_nodes])
            
            for a, b in matching:
                _, coord_a = z_defects[a]
                _, coord_b = z_defects[b]
                best_path, best_cost = diagonal_path_between(coord_a, coord_b)
                correction_paths[L]["Z"].append(best_path)
                for i in range(len(best_path) - 1):
                    midpoint = ((best_path[i][0] + best_path[i+1][0]) // 2,
                                (best_path[i][1] + best_path[i+1][1]) // 2)
                    for d_label, d_coord in grids_info[L]["di_coords"].items():
                        if d_coord == midpoint:
                            sorted_data_keys = sorted(grids_info[L]["di_coords"].keys(), key=lambda s: int(s.split('_')[1]))
                            index = sorted_data_keys.index(d_label)
                            qc.z(logical_registers[L][index])
                            corrected_qubits[L]["Z_correction"].append(d_label)
                            if verbose:
                                print(f"Lattice {L}: Z correction applied on {d_label} at {d_coord} (midpoint of {best_path[i]} and {best_path[i+1]})")
        else:
            if verbose:
                print(f"Lattice {L}: Z 오류 보정을 위한 Ax defect가 충분하지 않습니다. 감지된 Ax defects: {z_defects}")
            correction_paths[L]["unmatched_Z"].extend([anc_label for anc_label, _ in z_defects])
    
    if verbose:
        print("\nCorrected Qubits per Lattice:")
        for L_key, corrections in corrected_qubits.items():
            print(f"{L_key}: {corrections}")
    
        print("\nDetected Ancilla Defects per Lattice:")
        for L_key, anc_info in ancilla_defects_info.items():
            print(f"{L_key}: Az defects: {anc_info['Az_defects']}, Ax defects: {anc_info['Ax_defects']}")
    
    return qc, corrected_qubits, correction_paths


def generate_unmatched_syndrome_all(grids_info, correction_paths):
   
    unmatched_syndrome_all = {}
    for L in grids_info.keys():
        # X 오류 (Az ancilla)
        azi_keys = sorted(grids_info[L]["azi_coords"].keys(), key=lambda s: int(s.split('_')[1]))
        unmatched_X = correction_paths[L].get("unmatched_X", [])
        syndrome_X = "".join(['1' if key in unmatched_X else '0' for key in azi_keys])
        
        # Z 오류 (Ax ancilla)
        axi_keys = sorted(grids_info[L]["axi_coords"].keys(), key=lambda s: int(s.split('_')[1]))
        unmatched_Z = correction_paths[L].get("unmatched_Z", [])
        syndrome_Z = "".join(['1' if key in unmatched_Z else '0' for key in axi_keys])
        
        unmatched_syndrome_all[L] = {"X": syndrome_X, "Z": syndrome_Z}
    return unmatched_syndrome_all

def MWPM_paths(n, l, m, d, grids, correction_paths=None):
    import math
    import matplotlib.pyplot as plt

    def sort_points(points):
        cx = sum(p[0] for p in points) / len(points)
        cy = sum(p[1] for p in points) / len(points)
        return sorted(points, key=lambda p: math.atan2(p[1]-cy, p[0]-cx))
    
    fig, ax = plt.subplots(figsize=(15,15))
    grid_width = 2 * n
    grid_height = 2 * n
    
  
    for row in range(l):
        for col in range(m):
            grid_index = row * m + col + 1
            label_L = f"L_{grid_index}"
            ox = col * (grid_width + d)
            oy = row * (grid_height + d)
            
            di_coords = grids[label_L]["di_coords"]
            axi_coords = grids[label_L]["axi_coords"]
            azi_coords = grids[label_L]["azi_coords"]
            adj_axi = grids[label_L]["adj_axi"]
            adj_azi = grids[label_L]["adj_azi"]
            
           
            for key, (x, y) in di_coords.items():
                ax.scatter(x, y, facecolors='none', edgecolors='black', zorder=3, s=100)
                ax.text(x, y, key, color='black', ha='center', va='center', fontsize=10, zorder=4)
           
            for key, (x, y) in axi_coords.items():
                ax.scatter(x, y, color='yellow', s=80, zorder=2)
                ax.text(x, y, key, color='black', ha='center', va='center', fontsize=8, zorder=4)
           
            for key, (x, y) in azi_coords.items():
                ax.scatter(x, y, color='green', s=80, zorder=2)
                ax.text(x, y, key, color='black', ha='center', va='center', fontsize=8, zorder=4)
            
            for ax_label, (ax_x, ax_y) in axi_coords.items():
                adjacent = adj_axi.get(ax_label, [])
                if len(adjacent) >= 2:
                    pts = [di_coords[dq_label] for dq_label in adjacent]
                    pts = sort_points(pts)
                    xs, ys = zip(*pts)
                    ax.fill(xs, ys, color='lightgoldenrodyellow', alpha=0.5, zorder=0)
            
            for az_label, (az_x, az_y) in azi_coords.items():
                adjacent = adj_azi.get(az_label, [])
                if len(adjacent) >= 2:
                    pts = [di_coords[dq_label] for dq_label in adjacent]
                    pts = sort_points(pts)
                    xs, ys = zip(*pts)
                    ax.fill(xs, ys, color='palegreen', alpha=0.5, zorder=0)
            
            
            if correction_paths is not None and label_L in correction_paths:
                cp = correction_paths[label_L]
                
                for path in cp.get("X", []):
                    if len(path) > 1:
                        xs, ys = zip(*path)
                        ax.plot(xs, ys, color='red', linewidth=2, marker='o', label='X correction')
                
                for path in cp.get("Z", []):
                    if len(path) > 1:
                        xs, ys = zip(*path)
                        ax.plot(xs, ys, color='blue', linewidth=2, marker='o', label='Z correction')
                
                for anc_label in cp.get("unmatched_X", []):
                    
                    if anc_label in azi_coords:
                        coord = azi_coords[anc_label]
                        ax.scatter(coord[0], coord[1], marker='x', color='black', s=150, label='Unmatched X')
                
                for anc_label in cp.get("unmatched_Z", []):
                    if anc_label in axi_coords:
                        coord = axi_coords[anc_label]
                        ax.scatter(coord[0], coord[1], marker='x', color='black', s=150, label='Unmatched Z')
    
    total_width = m * (grid_width + d) - d
    total_height = l * (grid_height + d) - d
    ax.set_xlim(-1, total_width+1)
    ax.set_ylim(-1, total_height+1)
    ax.set_aspect('equal')
    ax.set_xticks(range(0, total_width+1))
    ax.set_yticks(range(0, total_height+1))
    ax.grid(True, which='both', linestyle='--', color='lightgray')
    plt.title(f"{l} x {m} lattice (n={n}, d={d}) with MWPM correction paths")
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
    plt.show()

def apply_unmatched_syndrome_correction(qc, unmatched_syndrome, grids_info, logical_registers, n, m):
    
    additional_corrections = {}
    
    def get_exclusive_data_qubits(L, anc_label, error_type):
        
        if error_type == "X":
            adj_dict = grids_info[L].get("adj_azi", {})
        else:
            adj_dict = grids_info[L].get("adj_axi", {})
        candidate_list = adj_dict.get(anc_label, [])
        exclusive = []
        
        all_connections = {}
        for a_label, d_list in adj_dict.items():
            for d in d_list:
                all_connections.setdefault(d, set()).add(a_label)
       
        for d in candidate_list:
            if all_connections.get(d, set()) == {anc_label}:
                exclusive.append(d)
        return exclusive

    for L in grids_info.keys():
        additional_corrections[L] = {"X": [], "Z": []}
       
        azi_keys = sorted(grids_info[L]["azi_coords"].keys(), key=lambda s: int(s.split('_')[1]))
        syndrome_X = unmatched_syndrome.get(L, {}).get("X", "")
        for i, anc_label in enumerate(azi_keys):
            if i < len(syndrome_X) and syndrome_X[i] == '1':
                exclusive_d = get_exclusive_data_qubits(L, anc_label, error_type="X")
                if not exclusive_d:
                    continue
                if 1 <= len(exclusive_d) <= 2:
                    d_target = exclusive_d[0]
                    sorted_data_keys = sorted(grids_info[L]["di_coords"].keys(), key=lambda s: int(s.split('_')[1]))
                    try:
                        index = sorted_data_keys.index(d_target)
                    except ValueError:
                        continue
                    qc.x(logical_registers[L][index])
                    additional_corrections[L]["X"].append(d_target)
                    print(f"Lattice {L}: Additional X correction applied on {d_target} using unmatched ancilla {anc_label}")
                else:
                    print(f"Lattice {L}: Skipping additional X correction for {anc_label} because exclusive data qubits > 2")
        
        
        axi_keys = sorted(grids_info[L]["axi_coords"].keys(), key=lambda s: int(s.split('_')[1]))
        syndrome_Z = unmatched_syndrome.get(L, {}).get("Z", "")
        for i, anc_label in enumerate(axi_keys):
            if i < len(syndrome_Z) and syndrome_Z[i] == '1':
                exclusive_d = get_exclusive_data_qubits(L, anc_label, error_type="Z")
                if not exclusive_d:
                    continue
                if 1 <= len(exclusive_d) <= 2:
                    d_target = exclusive_d[0]
                    sorted_data_keys = sorted(grids_info[L]["di_coords"].keys(), key=lambda s: int(s.split('_')[1]))
                    try:
                        index = sorted_data_keys.index(d_target)
                    except ValueError:
                        continue
                    qc.z(logical_registers[L][index])
                    additional_corrections[L]["Z"].append(d_target)
                    print(f"Lattice {L}: Additional Z correction applied on {d_target} using unmatched ancilla {anc_label}")
                else:
                    print(f"Lattice {L}: Skipping additional Z correction for {anc_label} because exclusive data qubits > 2")
    
    return qc, additional_corrections


def full_error_correction(qc, syndrome, grids_info, logical_registers, n, m):
    
    qc, corrected_qubits, correction_paths = error_correction(qc, syndrome, grids_info, logical_registers, n, m, verbose=True)
    
   
    unmatched_syndrome_all = generate_unmatched_syndrome_all(grids_info, correction_paths)
    print("\nUnmatched Syndrome per Lattice (both error types):")
    for L, syndrome_dict in unmatched_syndrome_all.items():
        print(f"{L}: X errors -> {syndrome_dict['X']}, Z errors -> {syndrome_dict['Z']}")
    
    
    qc, additional_corrections = apply_unmatched_syndrome_correction(qc, unmatched_syndrome_all, grids_info, logical_registers, n, m)
    
    return qc, corrected_qubits, correction_paths, unmatched_syndrome_all, additional_corrections



In [ ]:
# ================= Logical Operations and Interpretation =================

def xl(qc: QuantumCircuit, L_label: str, grids_info: dict, logical_registers: dict) -> QuantumCircuit:
    di_coords = grids_info[L_label]["di_coords"]
    unique_ys = sorted({coord[1] for coord in di_coords.values()})
    median_y = unique_ys[len(unique_ys) // 2]
    data_keys_sorted = sorted(di_coords.keys(), key=lambda k: int(k.split('_')[1]))
    data_reg = logical_registers[L_label]
    for i, dq_label in enumerate(data_keys_sorted):
        x, y = di_coords[dq_label]
        if y == median_y:
            qc.x(data_reg[i])
    return qc

def zl(qc: QuantumCircuit, L_label: str, grids_info: dict, logical_registers: dict) -> QuantumCircuit:
    di_coords = grids_info[L_label]["di_coords"]
    unique_xs = sorted({coord[0] for coord in di_coords.values()})
    median_x = unique_xs[len(unique_xs) // 2]
    data_keys_sorted = sorted(di_coords.keys(), key=lambda k: int(k.split('_')[1]))
    data_reg = logical_registers[L_label]
    for i, dq_label in enumerate(data_keys_sorted):
        x, y = di_coords[dq_label]
        if x == median_x:
            qc.z(data_reg[i])
    return qc

def get_central_block(di_coords: dict) -> list:
    xs = sorted({coord[0] for coord in di_coords.values()})
    ys = sorted({coord[1] for coord in di_coords.values()})
    median_x = xs[len(xs) // 2]
    median_y = ys[len(ys) // 2]
    dx = xs[1] - xs[0] if len(xs) > 1 else 0
    central_x = [val for val in [median_x - dx, median_x, median_x + dx] if val in xs]
    central_y = [val for val in [median_y - dx, median_y, median_y + dx] if val in ys]
    central_labels = []
    for label, (x, y) in di_coords.items():
        if x in central_x and y in central_y:
            central_labels.append(label)
    central_labels_sorted = sorted(central_labels, key=lambda lab: (di_coords[lab][1], di_coords[lab][0]))
    return central_labels_sorted

def lcx(qc: QuantumCircuit,
         L_control: str,
         L_target: str,
         logical_registers: dict,
         n: int) -> QuantumCircuit:
    control_reg = logical_registers[L_control]
    target_reg = logical_registers[L_target]
    for i in range(n * n):
        qc.cx(control_reg[i], target_reg[i])
    return qc

def Hl(qc: QuantumCircuit, L_label: str, grids_info: dict, logical_registers: dict) -> QuantumCircuit:
    data_reg = logical_registers[L_label]
    for qubit in data_reg:
        qc.h(qubit)
    return qc

def rot(qc: QuantumCircuit, L_label: str, grids_info: dict, logical_registers: dict) -> QuantumCircuit:
    di_coords = grids_info[L_label]["di_coords"]
    data_keys_sorted = sorted(di_coords.keys(), key=lambda k: (di_coords[k][1], di_coords[k][0]))
    N = len(data_keys_sorted)
    n = int(math.sqrt(N))
    if n * n != N:
        raise ValueError("데이터 큐빗의 총 수가 완전제곱수가 아닙니다.")
    unique_x = sorted({coord[0] for coord in di_coords.values()})
    unique_y = sorted({coord[1] for coord in di_coords.values()})
    mapping = [None] * N
    for i, label in enumerate(data_keys_sorted):
        x, y = di_coords[label]
        r = unique_y.index(y)
        c = unique_x.index(x)
        new_r = c
        new_c = n - 1 - r
        new_index = new_r * n + new_c
        mapping[i] = new_index
    data_reg = logical_registers[L_label]
    for i in range(N):
        while mapping[i] != i:
            j = mapping[i]
            qc.swap(data_reg[i], data_reg[j])
            mapping[i], mapping[j] = mapping[j], mapping[i]
    return qc

In [ ]:
# ================= Interpretation Functions with Dynamic Logical Basis =================

def interpret_logical_counts_from_full_counts_left(L_label: str, counts: dict, cd_len: int,
                                                     logical0_set: set, logical1_set: set) -> str:
    count0 = 0
    count1 = 0
    for outcome, cnt in counts.items():
        data_result = outcome[:cd_len]
        if data_result in logical0_set:
            count0 += cnt
        elif data_result in logical1_set:
            count1 += cnt
    if count0 > count1:
        return f"{L_label}: Logical 0 (0: {count0}, 1: {count1})"
    elif count1 > count0:
        return f"{L_label}: Logical 1 (0: {count0}, 1: {count1})"
    else:
        return f"{L_label}: Inconclusive (0: {count0}, 1: {count1})"

def interpret_balanced_constant_from_counts(counts: dict, n: int, num_lattices: int,
                                             logical0_set: set, logical1_set: set) -> dict:
    num_data = num_lattices - 1
    data_length = n * n * num_data
    balanced_count = 0
    constant_count = 0
    inconclusive_count = 0
    for outcome, cnt in counts.items():
        outcome_clean = outcome.replace(" ", "")
        left_data = outcome_clean[:data_length]
        segments = [left_data[i:i+n*n] for i in range(0, len(left_data), n*n)]
        if any(seg in logical1_set for seg in segments):
            balanced_count += cnt
        elif all(seg in logical0_set for seg in segments):
            constant_count += cnt
        else:
            inconclusive_count += cnt
    return {"balanced": balanced_count, "constant": constant_count, "inconclusive": inconclusive_count}

def interpret_composite_logical_state(counts: dict, n: int, num_lattices: int,
                                      logical0_set: set, logical1_set: set) -> dict:
    segment_length = n * n
    total_data_length = segment_length * num_lattices
    composite_counts = {}
    for outcome, cnt in counts.items():
        outcome_clean = outcome.replace(" ", "")
        left_data = outcome_clean[:total_data_length]
        if len(left_data) < total_data_length:
            continue
        segments = [left_data[i:i+segment_length] for i in range(0, total_data_length, segment_length)]
        segments_reversed = segments[::-1]
        composite_state_bits = ""
        for seg in segments_reversed:
            if seg in logical0_set:
                composite_state_bits += "0"
            elif seg in logical1_set:
                composite_state_bits += "1"
            else:
                composite_state_bits += "?"
        composite_state = f"logical |{composite_state_bits}>"
        composite_counts[composite_state] = composite_counts.get(composite_state, 0) + cnt
    return composite_counts


In [ ]:
# ================= Main Execution =================

n = 3
l = 3
m = 3


grids_info = plot_multi_grids(n, l, m, d=2)


stabilizers = generate_stabilizers(grids_info)


logical_registers = {}
x_ancilla_registers = {}
z_ancilla_registers = {}
for L_label, grid_data in grids_info.items():
    d_labels = list(grid_data["di_coords"].keys())
    num_d = len(d_labels)
    ql = QuantumRegister(num_d, name=f"ql_{L_label}")
    logical_registers[L_label] = ql
    ax_labels = list(grid_data["axi_coords"].keys())
    num_ax = len(ax_labels)
    qx = QuantumRegister(num_ax, name=f"qx_{L_label}")
    x_ancilla_registers[L_label] = qx
    az_labels = list(grid_data["azi_coords"].keys())
    num_az = len(az_labels)
    qz = QuantumRegister(num_az, name=f"qz_{L_label}")
    z_ancilla_registers[L_label] = qz

all_registers = []
for L_label in grids_info.keys():
    all_registers.extend([logical_registers[L_label],
                          x_ancilla_registers[L_label],
                          z_ancilla_registers[L_label]])
qc = QuantumCircuit(*all_registers)


stab_list_L1 = []
for key in sorted(stabilizers["L_1"]["x_stabilizers"].keys(), key=lambda k: int(k.split('_')[1])):
    stab_list_L1.append(stabilizers["L_1"]["x_stabilizers"][key])
for key in sorted(stabilizers["L_1"]["z_stabilizers"].keys(), key=lambda k: int(k.split('_')[1])):
    stab_list_L1.append(stabilizers["L_1"]["z_stabilizers"][key])
print("\nL_1 stabilizer list:")
for s in stab_list_L1:
    print(s)


In [ ]:
x_stabs_L1 = list(stabilizers["L_1"]["x_stabilizers"].values())
logical0_set_dynamic, logical1_set_dynamic = generate_logical_basis_sets(x_stabs_L1, n)

def enc(qc: QuantumCircuit) -> QuantumCircuit:
    for L_label in sorted(logical_registers.keys(), key=lambda k: int(k.split('_')[1])):
        stab_list = []
        for key in sorted(stabilizers[L_label]["x_stabilizers"].keys(), key=lambda k: int(k.split('_')[1])):
            stab_list.append(stabilizers[L_label]["x_stabilizers"][key])
        for key in sorted(stabilizers[L_label]["z_stabilizers"].keys(), key=lambda k: int(k.split('_')[1])):
            stab_list.append(stabilizers[L_label]["z_stabilizers"][key])
        encoding_circuit = synth_circuit_from_stabilizers(stab_list, allow_underconstrained=True)
        qc.compose(encoding_circuit, qubits=logical_registers[L_label][:], inplace=True)
    return qc

qc = enc(qc)


num_rounds = 1000
all_counts = []

sim = AerSimulator()

for i in range(num_rounds):
    
    qc_round = qc.copy()
    

    for L_label in grids_info.keys():
        qc_round = Hl(qc_round, L_label, grids_info, logical_registers)
        qc_round = rot(qc_round, L_label, grids_info, logical_registers)

    qc_round = inject_pauli_error(qc_round,logical_registers, error_probability=0.02)
    qc_round, syndrome = extract_syndrome(qc_round, grids_info, logical_registers, x_ancilla_registers, z_ancilla_registers)
    qc_round, corrected_qubits, correction_paths, unmatched_syndrome_all, additional_corrections = full_error_correction(qc_round, syndrome, grids_info, logical_registers, n, m)

    qc_round = inject_pauli_error(qc_round,logical_registers, error_probability=0.02)
    qc_round, syndrome = extract_syndrome(qc_round, grids_info, logical_registers, x_ancilla_registers, z_ancilla_registers)
    qc_round, corrected_qubits, correction_paths, unmatched_syndrome_all, additional_corrections = full_error_correction(qc_round, syndrome, grids_info, logical_registers, n, m)
    

    qc_round = inject_pauli_error(qc_round,logical_registers, error_probability=0.02)
    qc_round, syndrome = extract_syndrome(qc_round, grids_info, logical_registers, x_ancilla_registers, z_ancilla_registers)
    qc_round, corrected_qubits, correction_paths, unmatched_syndrome_all, additional_corrections = full_error_correction(qc_round, syndrome, grids_info, logical_registers, n, m)

    for L_label in grids_info.keys():
        qc_round = Hl(qc_round, L_label, grids_info, logical_registers)
        qc_round = rot(qc_round, L_label, grids_info, logical_registers)

   
    for L_label, data_reg in logical_registers.items():
        creg = ClassicalRegister(len(data_reg), name=f"cd_{L_label}")
        qc_round.add_register(creg)
        qc_round.measure(data_reg, creg)

    result = sim.run(qc_round, shots=1).result()
    counts = result.get_counts(qc_round)
    all_counts.append(counts)
    
    print(f"Round {i+1}: {counts}")

combined_counts = Counter()
for counts in all_counts:
    combined_counts.update(counts)

combined_counts = dict(combined_counts)

balanced_const_dict = interpret_balanced_constant_from_counts(combined_counts, n, l*m,
                                                                logical0_set=logical0_set_dynamic,
                                                                logical1_set=logical1_set_dynamic)
print("\nBalanced / Constant Interpretation:")
print(balanced_const_dict)

composite_state_dict = interpret_composite_logical_state(combined_counts, n, l*m,
                                                         logical0_set=logical0_set_dynamic,
                                                         logical1_set=logical1_set_dynamic)
print("\nComposite Logical State Interpretation:")
print(composite_state_dict)